In [1]:
from cryptography.hazmat.primitives.ciphers import algorithms
import common

## Problem 1

In [2]:
key = "140b41b22a29beb4061bda66b6747e14"
common.hex_str_to_bytes(key)

[20, 11, 65, 178, 42, 41, 190, 180, 6, 27, 218, 102, 182, 116, 126, 20]